In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 환경변수 및 패키지

In [22]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_ollama import OllamaEmbeddings, ChatOllama
import os
import re
import ollama

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

import pandas as pd

load_dotenv()

True

In [23]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'
OLLAMA_EMBEDDING_MODEL = 'bge-m3'

PINECONE_INDEX_NAME_LAW = 'law'
PINECONE_INDEX_NAME_RULE = 'rule'
PINECONE_INDEX_NAME_CASE = 'case'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 함수

In [43]:
# ==============
# 상용님 카테고리
# ==============

def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '월세전환': 3, '반전세': 2, '차임': 3, '월세': 2, '증액': 3, '월세전환': 3, '반전세': 2, 
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2, '감액': 2, '인상': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2
        },
        # [신설] 9. 임차권 승계 및 가족 (제9조 타겟팅)
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        }
    }


def categorize_content(content, top_k=3):
    """
    점수 기반 카테고리 분류 - 점수 높은 순으로 반환
    - 기본 top_k=3
    - 매칭 없으면 ['계약갱신','계약해지']로 폴백
    """
    category_keywords = get_law_category()
    category_scores = {}

    # 아주 간단한 정규화(공백 제거)만 해도 매칭률 올라감
    if content is None:
        content = ""
    content_norm = content.replace(" ", "")

    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            # keyword도 공백 제거해 비교(사전에 공백은 거의 없겠지만 안전)
            kw = keyword.replace(" ", "")
            count = content_norm.count(kw)
            score += count * weight
        if score > 0:
            category_scores[category] = score

    sorted_categories = sorted(category_scores.items(), key=lambda x: x[1], reverse=True)
    all_categories = [c for c, _ in sorted_categories]

    # ✅ 매칭 없으면 '기타' 말고 핵심 카테고리로 폴백
    if not all_categories:
        all_categories = ['계약갱신', '계약해지']

    return all_categories[:top_k]


# =====================
# 참조조항을 추출하는 함수
# =====================

def extract_articles_from_docs(documents):
    '''검색된 문서들(documents)에서 aritcle 정보를 추출하여 리스트로 변환
    Returns : 중복이 제거된 조항 리스트 ex : ['제4조', '제16조']'''
    articles = []
    for doc in documents:
        article = doc.metadata.get('article', '조항없음')
        
        # 「」제거. 문자를 list로 분리
        if article != '조항없음':
            article = article.replace('「','').replace('」','')
            article_list = article.split(', ')
            articles.extend(article_list)
            
    # 중복 제거 및 정렬
    unique_articles = list(set(articles))
    unique_articles.sort(key=lambda x : int(x[:-1]))
    final_articles = ['제'+article for article in unique_articles] 
    return '주택임대차보호법' + ','.join(final_articles)


# ====================================
# 검색된 document를 텍스트로 변환하는 함수
# ====================================

def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])


# ===========================
# title+text 기반의 rerank함수
# ===========================

import cohere
import numpy as np

def rerank_by_title(query: str, documents: list, top_k: int = 4):
    '''
    cohere의 rerank API를 사용한 문서 재정렬
    '''
    # cohere client 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    # 0건이면 rerank 호출하지 말고 그대로 반환
    if not documents:
        return []

    # top_k도 0이면 그대로
    if not top_k or top_k <= 0:
        return documents

    docs_text = [
        (doc.metadata.get("title", "") or "") + " " + (doc.page_content or "")
        for doc in documents
    ]

    # documents가 비어있지 않아도, docs_text가 전부 빈문자면 Cohere가 싫어할 수 있어서 방어
    if not any(t.strip() for t in docs_text):
        return documents[:top_k]

    results = co.rerank(
        model="rerank-multilingual-v3.0",
        query=query,
        documents=docs_text,
        top_n=min(top_k, len(docs_text)),
    )

    idxs = [r.index for r in results.results]
    return [documents[i] for i in idxs]


In [38]:
def get_token_usage(llm, llm_response):
    """
    langchain_openai.ChatOpenAI 응답에서 토큰 사용량 추출.
    - 우선: llm_response.usage_metadata (네 로그에서 실제 존재)
    - 차선: llm_response.response_metadata["token_usage"]
    - 그 외: None
    """
    # OpenAI가 아니면 None (Ollama 등)
    if llm.__class__.__module__ != "langchain_openai.chat_models.base":
        return None

    # 1) usage_metadata
    usage_meta = getattr(llm_response, "usage_metadata", None)
    if isinstance(usage_meta, dict) and usage_meta:
        return {
            "input_tokens": usage_meta.get("input_tokens"),
            "output_tokens": usage_meta.get("output_tokens"),
            "total_tokens": usage_meta.get("total_tokens"),
        }

    # 2) 일부 케이스: response_metadata 안 token_usage
    meta = getattr(llm_response, "response_metadata", None) or {}
    token_usage = meta.get("token_usage")
    if isinstance(token_usage, dict) and token_usage:
        # token_usage는 보통 prompt_tokens / completion_tokens 형태일 수 있음
        prompt = token_usage.get("prompt_tokens") or token_usage.get("input_tokens")
        completion = token_usage.get("completion_tokens") or token_usage.get("output_tokens")
        total = token_usage.get("total_tokens")
        if total is None and prompt is not None and completion is not None:
            total = prompt + completion
        return {
            "input_tokens": prompt,
            "output_tokens": completion,
            "total_tokens": total,
        }

    return None



import re

def sanitize_for_upstage(text: str, max_len: int = 500) -> str:
    """Upstage embedding input 안전 정제: str 보장, 공백/개행 정리, 길이 제한"""
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 이상한 프롬프트 잔재 제거(가끔 '질문 :' 같은 prefix가 섞임)
    text = re.sub(r"^\s*(질문\s*:|question\s*:|q\s*:)\s*", "", text, flags=re.IGNORECASE)

    # 개행/탭 과다를 단일 공백으로
    text = re.sub(r"\s+", " ", text).strip()

    # 너무 길면 자르기
    if len(text) > max_len:
        text = text[:max_len].strip()

    return text

def safe_embedding_query(normalized_query: str, original_query: str) -> str:
    q = sanitize_for_upstage(normalized_query)
    if not q:  # 빈 문자열이면 원문으로 폴백
        q = sanitize_for_upstage(original_query)
    return q


def retrieve_law_docs_by_priority(
    law_db: PineconeVectorStore,
    embedding: UpstageEmbeddings,
    query_for_embed: str,
    categories: list,
    k_total: int = 4,
    k_each: int = 4,
    priorities: list = [1,2,3,4,5,6,7],
) -> list:
    """
    law 인덱스에서 priority=1부터 순차적으로 확인하며 문서를 누적 수집.
    - embedding은 1회만 생성
    - priority가 낮을수록(1이 가장 중요) 우선적으로 채움
    """
    # Upstage에 넣는 query는 반드시 정제된 값을 사용
    clean_q = safe_embedding_query(query_for_embed, query_for_embed)
    qvec = embedding.embed_query(clean_q)

    collected = []
    seen = set()

    for p in priorities:
        flt = {"priority": {"$eq": p}}
        if categories:  # None 또는 []면 category 조건을 아예 넣지 않음
            flt["category"] = {"$in": categories}

        docs = law_db.similarity_search_by_vector(
            qvec,
            k=k_each,
            filter=flt
        )

        for d in docs:
            # 중복 제거(간단 key)
            key = (d.metadata.get("id"), d.metadata.get("source"), d.page_content[:120])
            if key in seen:
                continue
            seen.add(key)
            d.metadata.setdefault("priority", p)
            collected.append(d)

        if len(collected) >= k_total:
            break

    return collected[:k_total]


# 1. 비용계산

In [25]:
OPENAI_PRICES = {
    "gpt-4o-mini": {
        "input": 0.15 / 1000000,
        "output": 0.60 / 1000000,
    },
    "gpt-4.1-mini": {
        "input": 0.40 / 1000000,
        "output": 1.60 / 1000000,
    },
    "gpt-5-mini": {
        "input": 0.25 / 1000000,
        "output": 2.00 / 1000000,
    },
    "gpt-5-nano": {
        "input": 0.05 / 1000000,
        "output": 0.40 / 1000000,
    },
    "gpt-4.1-nano": {
        "input": 0.10 / 1000000,
        "output": 0.40 / 1000000,
    },
}


def calculate_openai_cost(model_name: str, token_usage: dict) -> float | None:
    if token_usage is None:
        return None
    price = OPENAI_PRICES.get(model_name)
    if not price:
        return None

    return (
        token_usage["input_tokens"]  * price["input"] +
        token_usage["output_tokens"] * price["output"]
    )

# 2. LLM

## index law

In [39]:
# 재사용 가능한 함수로 만들기

def ask_with_reference(query:str, k:int=4, top_k:int=4):
    'query, 표준화된 query, 답변, 참조사항 출력'
    
    # 1. llm, embedding
    
    ### OPENAI ###
    # llm = ChatOpenAI(model='gpt-4o-mini')
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    # llm = ChatOpenAI(model='gpt-4.1-nano')
    # llm = ChatOpenAI(model='gpt-5-mini')
    # llm = ChatOpenAI(model='gpt-5-nano')
    
    ### OLLAMA ###
    llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOllama(model="exaone3.5:7.8b", temperature=0.1)
    # llm = ChatOllama(model="bnksys/yanolja-eeve-korean-instruct-10.8b", temperature=0.1)
    # llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.1)
    # llm = ChatOllama(model="", temperature=0.1)
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)
    ol_embedding = OllamaEmbeddings(model=OLLAMA_EMBEDDING_MODEL)

    # 2. 업로드한 벡터 DB에서 가져올 때
    law_database= PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)

    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

    case_database = PineconeVectorStore(
        embedding=up_embedding,
        # embedding=ol_embedding,
        index_name=PINECONE_INDEX_NAME_CASE)
    
    # 3. keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 보증금 반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # [행위 - 거주/계약]
        "계약서 -> 임대차계약증서", "특약 -> 특약사항",
        "이사 -> 주택의인도", "전입 -> 주민등록/대항력",
        "살고 -> 점유", "거주 -> 점유",
        "나가라 -> 계약갱신거절/명도", "나가라고 -> 계약갱신거절/명도", "비워 -> 명도", "쫓겨 -> 명도/대항력",
        "방 빼 -> 계약해지", "나갈게 -> 계약해지", "해지 -> 계약해지",
        "연장 -> 계약갱신", "더 살 -> 계약갱신", "재계약 -> 계약갱신",
        "수리 -> 수선의무/필요비", "고쳐 -> 수선의무", "물 새 -> 누수", "곰팡이 -> 하자",
        "청소비 -> 원상회복비용", "원상복구 -> 원상회복", "한달 남 -> 기간만료",
        "1달 남 -> 기간만료", "계약 끝 -> 기간만료",
    ]

    prompt = ChatPromptTemplate.from_template(f'''아래 질문에서 단어만 사전을 기준으로 치환하세요.문장 구조, 시제, 의문형 여부는 변경하지 마세요
    사전 : {keyword_dict}
    질문 : {{question}}
    출력은 질문 문장만 하세요''')

    keyword_chain = prompt | llm | StrOutputParser()
    
    # 4. 질문을 표준화
    normalized_query = keyword_chain.invoke({'question':query}) # 출력함

    # 5. 관련문서 검색
    # retriever = law_database.as_retriever(search_kwargs = {'k':k, 'filter':{'category':{'$in':categorize_content(query)}}})
    retriever = law_database.as_retriever(search_kwargs={'k': k})

    # 관련문서  rerank
    embed_query = safe_embedding_query(normalized_query, query)
    categories = categorize_content(query)
    
    retrieved_docs = retrieve_law_docs_by_priority(
        law_db=law_database,
        embedding=up_embedding,
        query_for_embed=embed_query,
        # categories=categories,
        categories=None,
        k_total=k,         # 최종 문서 수는 기존 k를 따름
        k_each=max(k, 4),  # 각 priority에서 몇 개씩 볼지(넉넉히)
        priorities=[1,2,3,4,5,6,7]
    )
    reranked_docs = rerank_by_title(embed_query, retrieved_docs, top_k) # 최종 documents
    referenced_articles = extract_articles_from_docs(reranked_docs) # 참조조항 출력

    template = f"""
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

    [지시사항]
    1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
    2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
    3. **답변 구조**:
       - 결론부터 명확히(가능/불가능) 제시
       - 📖 법적 근거(제N조) 상세 설명
       - 사용자 상황에 맞는 조언
    4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
    5. **❗ 면책**: 법적 효력이 없음을 밝히세요.

    [참고 법령]
    {{context}}
    질문 : {{query}}
    """
    prompt = ChatPromptTemplate.from_template(template)
    # prompt_chain = prompt | llm | StrOutputParser()
    # result = prompt_chain.invoke({'context':format_documents(retrieved_docs),
                                  # 'query':query})
    prompt_chain = prompt | llm
    llm_response = prompt_chain.invoke({'context':format_documents(reranked_docs),
                                        'query':query})
    result = llm_response.content
    
    
    print('원본 질문 :', query)
    print('표준화된 질문 :', normalized_query)
    print('='*90)
    print('\n✅', result)
    # print('\n📌 참조사항 :', referenced_articles)
    # print('\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗')
    
    token_usage = get_token_usage(llm, llm_response)
    cost = calculate_openai_cost("gpt-5-nano", token_usage)
    # gpt-4o-mini  /  gpt-4.1-mini  /  gpt-4.1-nano  /  gpt-5-mini  /  gpt-5-nano
    
    print("\n📊 토큰 사용량")
    if token_usage is None:
        print("- Input tokens  : None")
        print("- Output tokens : None")
        print("- Total tokens  : None")
    else:
        print(f"- Input tokens  : {token_usage['input_tokens']}")
        print(f"- Output tokens : {token_usage['output_tokens']}")
        print(f"- Total tokens  : {token_usage['total_tokens']}")
        
    print("\n💰 추정 비용")
    print(cost if cost is not None else "None")
    
    # ======================================================
    # 🔴 🔴 🔴 여기에 아래 디버그 코드 전부 복붙 🔴 🔴 🔴
    # ======================================================

    print("[DEBUG] retrieved_docs:", len(retrieved_docs))
    print("[DEBUG] reranked_docs:", len(reranked_docs))

    # ======================================================
    # 🔴 🔴 🔴 여기까지 🔴 🔴 🔴
    # ======================================================

### 모델 비교

#### exaone3.5:2.4b

In [77]:
%%time
# 사용예시 : exaone3.5:2.4b
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 집주인이 계약 끝나기 한 달 전에 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?

✅ ### 결론
**불가능** - 임차인은 계약 갱신 요구권을 행사할 수 있으며, 이 경우 집주인의 요구를 무조건적으로 따르지 않아도 됩니다.

### 법적 근거 (제6조의3)
- **제6조의3 제2항**에 따르면, 임대인은 임차인이 계약 갱신을 요구할 경우 정당한 사유 없이 거절할 수 없습니다. 특히, 다음과 같은 경우를 제외하고는 계약 갱신을 거절할 수 없습니다:
  - **제1호**: 임차인이 2기의 차임액에 해당하는 금액에 이르도록 차임을 연체한 경우
  - **제2호**: 임차인이 거짓이나 부정한 방법으로 임차한 경우
  - **제3항**: 임대인과 임차인 간에 합의하여 상당한 보상을 제공한 경우
  - **제4항**: 임차인이 주택을 전대(轉貸)한 경우
  - **제5항**: 임차인이 고의나 중대한 과실로 주택을 파손한 경우
  - **제6항**: 주택이 멸실되어 임대차 목적을 달성할 수 없는 경우
  - **제7항**: 임대인이 철거 또는 재건축 계획을 가지고 있는 경우 (구체적인 고지 및 동의 필요)
  - **제8항**: 임대인이 직접 거주하려는 경우
  - **제9항**: 임차인의 중대한 의무 위반 또는 임대차 유지가 어려운 중대한 사유가 있는 경우

### 사용자 상황에 맞는 조언
- **계약 갱신 요구권 활용**: 집주인이 계약 갱신을 요구하지 않고 무조건 나가라고 요구하더라도, **제6조의3 제2항**에 따라 임차인은 계약 갱신을 요구할 수 있습니다. 특히, 위에서 언급된 정당한 거절 사유에 해당하지 않는 한, 집주인의 요구를 무조건적으로 따르지 않아도 됩니다.
- **증거 준비**: 만약 집주인이 특정 사유를 제시하지 않고 무조건 나가라고 요구한다면, 계약 갱신 요구의 정당성을 입증

In [44]:
%%time
# 사용예시 : exaone3.5:2.4b
# query 2 : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?(주택임대차보호법 제7조의2 보증금 ↔ 월세 전환 비율, 시행령 제9조 언급해야함)

ask_with_reference('보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?')

원본 질문 : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
표준화된 질문 : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?

✅ ### 결론
**불가능**: 보증금을 줄이고 월세로 전환하려는 계획은 법적 제약이 있어 실행하기 어렵습니다. 과도한 월세 인상은 법 위반 가능성이 높습니다.

### 법적 근거 (제7조의2 및 제10조의2)
1. **제7조의2 (월차임 전환 시 산정률의 제한)**
   - **내용**: 보증금을 월 단위의 차임으로 전환할 때, 전환되는 금액은 다음 중 낮은 비율을 곱한 월차임 범위 내에서만 가능합니다. 구체적으로는:
     - **은행 대출금리 및 지역 경제 여건 고려 비율**
     - **한국은행 기준금리에 대통령령으로 정해진 이율을 더한 비율**
   - **적용 범위**: 이 규정은 월세 전환 시 적용되는 기본적인 상한선을 설정하여 과도한 인상을 방지합니다.

2. **제10조의2 (초과 차임 등의 반환청구)**
   - **내용**: 임차인이 이미 지급한 차임(월세 포함)이 법에서 정한 증액 비율을 초과한 경우, 초과 지급된 금액을 반환받을 수 있습니다.
   - **적용 범위**: 이 조항은 이미 과도하게 지급된 월세에 대한 반환 청구를 보장하지만, 전환 과정에서의 법적 상한선 준수는 여전히 필수적입니다.

### 사용자 상황에 맞는 조언
- **과도한 월세 인상 금지**: 현재 법률에 따르면, 보증금을 월세로 전환하는 과정에서 월차임의 상한선을 초과하는 인상은 금지되어 있습니다. 따라서 월세를 과도하게 인상하려는 계획은 즉시 중단해야 합니다.
- **법률 준수 필요성**: 월세 인상 시에도 **제7조의2**에 따른 증액 비율을 엄격히 준수해야 합니다. 예를 들어, 기존 차임의 20분의 1을 초과하는 증액은 제한됩니다 (제7조 제2항 참조).
- **합의 및 투명성**: 임차인과의 합의 과정에서 투명성을 유지하고, 

#### exaone3.5:7.8b

In [11]:
%%time
# 사용예시 : exaone3.5:7.8b
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 계약 기간 만료 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 명도를 요구합니다. 그래도 명도해야 하나요?

✅ **결론**: **불가능**합니다. 집주인이 계약 종료 한 달 전에 무조건적으로 임차인을 내보내려고 하더라도, **묵시적 갱신** 조항에 따라 임차인은 계속 거주할 권리가 있습니다.

**📖 법적 근거**:
- **주택임대차보호법 제6조(계약의 갱신)** ① 조항에 따르면, 임대인이 임대차기간 종료 6개월 전부터 2개월 전까지 갱신거절 통지를 하지 않거나 계약 조건 변경 없이 유지하면, 임대차는 동일한 조건으로 자동 갱신됩니다.
- **주택임대차보호법 제6조의2(묵시적 갱신의 경우 계약의 해지)** ① 조항에 따르면, 묵시적으로 갱신된 계약 하에서는 임차인은 언제든지 임대인에게 계약 해지를 통지할 수 있으며, 통지 후 3개월이 지나면 효력이 발생합니다. 하지만 임대인이 이미 새로운 임차인을 구했다는 이유만으로 기존 임차인을 무조건적으로 내보내는 것은 법적으로 허용되지 않습니다.

**사용자 상황에 맞는 조언**:
- 집주인이 새로운 임차인을 구했다고 하더라도, 계약 종료 전에 명확한 해지 통지가 없었다면 묵시적 갱신이 적용됩니다. 따라서 임차인은 현재 거주를 계속할 권리가 있습니다.
- 만약 집주인이 계약 해지를 요구한다면, 임차인은 해당 통지에 대해 법적 근거를 요구하고, 필요하다면 법률 자문을 구하거나 관련 기관에 문의하는 것이 좋습니다.

**🚨 위험 경고**:
- 집주인의 무조건적인 내보내기 요구는 **법 위반**일 수 있으며, 이로 인해 발생하는 손해에 대해 임차인은 법적 구제를 청구할 수 있습니다. 특히, 임대인이 정당한 사유 없이 계약을 해지하려고 할 경우, 임차인은 손해배상을 청구할 수 있습니다 (주택임대차보호법 제6조의3 제5항 참조).

**❗ 면책**:
- 이 조언은 

#### gpt-4o-mini

In [81]:
%%time
# 사용예시 : gpt-4o-mini
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 계약 기간만료 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?

✅ 결론적으로, **나가실 필요가 없습니다**.

📖 법적 근거: 
주택임대차보호법 제6조에 따르면, 임대인은 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 임차인에게 계약 갱신 거절 의사를 통지해야 합니다. 만약 임대인이 이러한 통지를 하지 않았다면, 계약은 묵시적으로 연장된 것으로 간주되며(제6조 제1항), 계약 기간은 2년으로 봅니다(제6조 제2항).

특히, 임대인이 계약 기간 종료 전 한 달 동안에 임차인에게 갱신 거절을 통지하지 않았다면, 임차인은 계속해서 해당 주택을 임차할 권리가 있습니다.

따라서, 집주인이 이미 다음 세입자를 구했다 하더라도 법적으로 정당한 절차를 거치지 않은 이상 임차인은 퇴거 요구에 응할 의무가 없습니다.

💡 조언:
1. 집주인과의 대화에서 본인의 권리를 주장하시고, 법적 근거를 참고하여 상황을 설명하십시오.
2. 만약 집주인이 계약을 위반하며 강제로 나가라고 한다면, 법적인 대응을 준비하는 것도 고려해 보아야 합니다. 필요한 경우 법률 전문가의 상담을 받는 것도 좋은 방법입니다.
3. 이후의 상황 전개에 따라 법적 절차를 통해 권리를 보호할 수 있습니다.

🚨 위험 경고:
가장 중요한 점은 임대차 계약서와 법적 권리를 제대로 파악하고 준수하는 것입니다. 불리한 특약이나 법 위반 사항이 있다면, 집주인이 주장할 수 있는 법적 근거에 주의하십시오.

❗ 면책: 본 답변은 법적 자문이 아니며, 구체적인 사안에 따라 추가 상담이 필요할 수 있습니다.

📊 토큰 사용량
- Input tokens  : 1711
- Output tokens : 416
- Total tokens  : 2127

💰 추정 비용
0.00050625
CPU times: total:

#### gpt-4.1-mini

In [79]:
%%time
# 사용예시 : gpt-4.1-mini
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 기간만료 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 계약갱신거절/명도 합니다. 그래도 계약갱신거절/명도 해야 하나요?

✅ - 결론: **무조건 나가야 할 의무는 없으며, 계약 기간이 끝나기 전 임대인이 임차인을 강제로 퇴거시키는 것은 불가능합니다.**

- 📖 법적 근거  
  - 주택임대차보호법 제6조 제1항에 따르면 임대차 계약 기간이 끝나기 6개월 전부터 2개월 전 사이에 임대인이 임차인에게 '갱신 거절'의 통지를 해야 합니다. 이 기간 내에 갱신 거절 통지를 하지 않으면 기존 조건으로 자동 갱신(묵시적 갱신)된 것으로 봅니다.  
  - 질문 상황에서는 집주인이 계약 종료 한 달 전에 갱신 거절 통지를 하는 것인데, 이는 법상 ‘계약 종료 2개월 전’ 통지 의무를 위반한 것입니다.  
  - 또한 임대차 기간이 끝나기 전까지는 계약이 유효하며, 임차인은 계속 주택에 거주할 권리가 있습니다(제4조 제2항). 임대인이 다음 세입자를 구했다고 하더라도, 기존 임차인이 계약 종료일까지 거주할 권리를 침해할 수 없습니다.

- 사용자 상황에 맞는 조언  
  - 계약 종료 1개월 전은 법정 통지기간(6개월~2개월 전)을 지나친 시점으로, 임대인이 무조건 나가라고 요구할 법적 근거가 없습니다.  
  - 임대인이 계약 조건 변경이나 갱신 거절을 적법한 시기에 통지하지 않았다면, 자동으로 계약은 갱신된 것으로 보아 2년간 거주할 권리가 있습니다.  
  - 따라서 기존 임대차 계약 기간이 끝날 때까지 거주할 수 있으며, 임대인이 임의로 퇴거를 강요할 경우 이는 불법입니다.  
  - 만약 강제로 퇴거를 요구하거나 위협하는 상황이라면 즉시 법률 상담을 받거나 관할 경찰서에 도움을 요청하는 것이 안전합니다.

- 🚨 위험 경고  
  - 임대인이 법정 갱신거절 통지를 ‘2개월 전’까지 하지 않은 경우,

#### gpt-5-mini

In [17]:
%%time
# 사용예시 : gpt-5-mini
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 계약 끝나기 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 계약갱신거절/명도 합니다. 그래도 나가야 하나요?

✅ 결론 — 불가능 (무조건 나가야 할 법적 의무는 없음)
- 임대차계약 기간이 끝나기 전(계약 기간 만료일이 도래하기 전)에 임대인이 일방적으로 “무조건 나가라”라고 요구해도, 특별한 사유(임차인의 중대한 채무불이행 등)가 없는 한 임차인은 계약 만료일까지 계속 거주할 권리가 있습니다.

📖 법적 근거(제조항) 상세 설명
1) 주택임대차보호법 제4조 제2항
- “임대차기간이 끝난 경우에도 임차인이 보증금을 반환받을 때까지는 임대차관계가 존속되는 것으로 본다.”
  → 계약기간이 끝나더라도 보증금을 돌려받기 전까지는 점유권이 유지된다는 취지로, 기간 도과 전이라면 더더욱 임차인의 점유권은 보호됩니다.

2) 주택임대차보호법 제6조(계약의 갱신) 제1항·제2항
- 임대인이 갱신거절 통지를 하지 아니하면 묵시적 갱신(종전 조건으로 재계약)으로 본다. 임대인은 임차인이 갱신을 요구할 경우 정당한 사유 없이 거절할 수 없다(제6조의3 제1항).
  → 임대인이 “다음 세입자를 구했다”는 이유만으로 계약 만료 전에 즉시 퇴거를 요구하는 것은 위 법리와 맞지 않습니다. 또한 임대인이 갱신을 거절하려면 정당한 사유(법에서 정한 예외들)가 있어야 합니다(제6조의3).

3) 주택임대차보호법 제6조의3(계약갱신요구 등)
- 임대인은 임차인이 계약갱신을 요구할 경우 정당한 사유 없이 거절할 수 없음.
- 다만 임차인이 2기 차임 연체, 전대, 고의·중대한 파손, 임대인의 직계존속·직계비속의 실제 거주 등 법에서 정한 예외가 있는 경우에는 갱신 거절 가능함(제6조의3 제1항 각 호).
- 또한 임대인이 임차인의 갱신요구를 정당한 사유 없이 거절하였음에도 갱신되었을 기간 만료 전에 제3자에게 임대한 경우 손해배

#### gpt-5-nano

In [19]:
%%time
# 사용예시 : gpt-5-nano
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 그래도 나가야 하나요?

✅ 다음은 제시된 법령을 바탕으로 한 해석입니다. 핵심 키워드: 묵시적 갱신, 계약갱신, 갱신거절, 전대, 해지, 분쟁조정.

결론
- 현재 상황에서 바로 나가야 할 법적 의무가 있다고 보긴 어렵습니다. 원칙상 계약종료 시점까지는 임차인으로서의 거주권이 보호될 수 있으며, 갱신 여부와 해지통지는 법적 요건에 따라 결정됩니다. 다만 상황에 따라 달라질 수 있으니, 구체적으로는 계약서 내용과 실제 통지 여부를 확인해야 합니다.

- 중요한 포인트: 계약 종료 1개월 남은 시점에 귀하가 스스로 즉시 이사를 해야 한다고 강제하는 것은 일반적인 법적 근거가 약합니다. 다만 임대인이 갱신 여부를 둘러싼 법적 절차를 어기거나, 임차인이 갱신을 요구할 때 정당한 사유 없이 거절하는 경우 등 예외가 존재합니다. 아래 근거를 참고해 상황을 점검하시길 권합니다.

📖 법적 근거(주요 조문 요지)
- 제6조(계약의 갱신)
  - 임대인이 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 갱신거절 통지를 하지 않거나 계약조건 변경을 하지 않으면, 그 기간이 끝난 때에 전 임대차와 동일한 조건으로 다시 임대차한 것으로 본다. 즉, 기간 종료 시점에 자동으로 2년 간의 갱신으로 보는 묵시적 갱신이 성립할 수 있다.
  - 임차인이 기간 만료 2개월 전까지 통지하지 않더라도 같은 효과가 적용될 수 있다.
  - 요지: 종료 시점까지 임대인의 의사표시가 없으면 대개 갱신으로 보며, 임차인은 원칙적으로 2년 간의 동일조건으로 지속될 수 있습니다.

- 제6조의2(묵시적 갱신의 경우 계약의 해지)
  - 묵시적 갱신이 성립한 경우라도 임차인은 임대인에게 계약해지를 통지할 수 있습니다. 해지효력은 통지받은 날로부터 3개월이 지난 때 발생합니다.
  - 요지: 묵시적 갱신이 성립했다면 임차인은 일정 기간을 두고 해지 의

#### bnksys/yanolja-eeve-korean-instruct-10.8b

In [13]:
%%time
# 사용예시 : bnksys/yanolja-eeve-korean-instruct-10.8b
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 임대차계약 만료 1개월 전에 새로운 임차인을 찾았다며 무조건 퇴거하라고 요구할 수 있나요?

✅ 
**답변:** **[가능] 네, 임대차 보호법에 따라 갱신 요구권을 행사할 수 있습니다.**

**법적 근거:** 주택임대차보호법 제6조의3(계약갱신 요구 등)에 따르면, 임차인은 계약 만료 전 2개월 이내에 계약갱신을 요구할 권리가 있으며, 임대인은 정당한 사유 없이 거절하지 못합니다. 이 경우 집주인이 다음 세입자를 구했다고 해서 정당한 사유로 간주되지 않습니다.

**상황에 맞는 조언:** 귀하의 상황에 따라 다음과 같은 조치를 취할 수 있습니다:

1. **계약갱신요구권 행사:** 계약 만료 전 2개월 이내에 임대인에게 계약갱신을 요구하세요. 이는 집주인이 다음 세입자를 구했다고 해도 유효합니다.
2. **차임 연체 확인:** 차임을 연체한 적이 없는지 확인하세요. 만약 연체가 있다면, 이는 정당한 거절 사유가 될 수 있습니다.
3. **대항력 확보:** 대항력을 확보하고 있는지 확인하세요. 대항력은 임차인이 임대인의 동의 없이 목적 주택의 전부 또는 일부를 전대(轉貸)하거나 고의나 중대한 과실로 파손한 경우 집주인이 계약갱신을 거절할 수 없도록 합니다.
4. **법적 도움 구하기:** 귀하의 권리가 침해되었다고 생각되면 법률 전문가와 상담하여 법적 도움을 받으세요.

**🚨 위험 경고:** 임대인의 정당한 거절 사유를 무시하고 나가지 않으면, 강제집행이나 기타 법적 조치에 직면할 수 있습니다. 항상 귀하의 권리를 이해하고 법적 조언을 구하세요.

**❗ 면책:** 이 답변은 일반적인 정보만을 제공하며 법적 조언으로 간주되어서는 안 됩니다. 귀하의 구체적인 상황에 맞는 법률 자문을 받으시기 바랍니다.

**참고 법령:** 주택임대차보호법 제6조의3(계약갱신 요구 등) 및 제4조(임대차기간 등).

📊 

In [15]:
%%time
# 사용예시 : qwen2.5:7b-instruct
# query 1 : 계약이 1달 남은 시점에서 집주인이 갑자기 나가래요(주택임대차보호법 제 6조(계약의 갱신) 언급해야함)

ask_with_reference('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

원본 질문 : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
표준화된 질문 : 임대인이 기간만료 한 달 전인데, 이미 다음 임차인을 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?

✅ ### 결론
이 상황에서는 **불가능**합니다.

### 📖 법적 근거 (제6조)
1. **계약 갱신의 의무**: 제6조는 임대차기간이 끝나기 6개월 전부터 2개월 전까지의 기간에 임차인에게 갱신거절이나 계약조건 변경을 통지하지 않으면, 그 기간이 끝난 때에 전 임대차와 동일한 조건으로 다시 임대차가 성립된 것으로 보는 내용을 담고 있습니다. (제6조 ①)
2. **계약 갱신의 존속기간**: 제6조 ②항은 이 경우 임대차의 존속기간이 2년으로 본다는 것을 명시하고 있습니다.

### 상황 분석
주어진 질문에서 집주인이 계약 끝나기 한 달 전에 무조건 나가라고 하는 것은 법적으로 문제가 있습니다. 제6조는 임대인(집주인)에게 계약 갱신의 의무를 부과하고 있으며, 이 경우 임차인(세입자)은 언제든지 계약을 해지할 수 있는 권리가 있습니다.

### 조언
1. **법적 검토**: 먼저 법률 전문가와 상담하여 정확한 법적 판단을 받으시는 것이 좋습니다.
2. **통지**: 집주인에게 제6조에 따른 갱신거절 통지를 요구하시고, 그 통지를 받은 날부터 3개월이 지나면 계약 해지의 효력이 발생한다는 점을 명확히 알리세요.
3. **차임 및 보증금**: 임대인과 협상하여 차임이나 보증금 등에 대한 조정을 요구할 수 있습니다.

### 🚨 위험 경고
불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고합니다. 집주인이 계약 끝나기 한 달 전에 무조건 나가라고 하는 것은 제6조의 강행규정을 위반하는 것이므로, 이는 불법이며 법적 대응이 필요할 수 있습니다.

### ❗ 면책
위 내용은 법률적 조언으로서 법적 효력을 갖추지 않으며, 실제 상황에 따른 법적 판단과 조치를 위해 전문적인 법률 서비스를 이용하시기 

## index 추가

In [29]:
from typing import Dict, List, Tuple, Optional

# =======================================
# 유틸: 문서 중복 제거 (내용+메타 기반 간단 키)
# =======================================

def dedupe_docs(docs: List[Document]) -> List[Document]:
    seen = set()
    out = []
    for d in docs:
        key = (
            d.metadata.get("id") or d.metadata.get("source") or "",
            d.metadata.get("article") or "",
            d.metadata.get("priority") or "",
            d.page_content[:200],  # 너무 길면 부담, 앞부분만
        )
        if key in seen:
            continue
        seen.add(key)
        out.append(d)
    return out


# =======================================
# 유틸: priority별로 여러 벡터스토어에서 검색
# =======================================

def retrieve_by_priority(
    query: str,
    vectorstores: Dict[str, "PineconeVectorStore"],  # {"law": law_db, "rule": rule_db, "case": case_db}
    categories: List[str],
    priorities: List[int] = [1,2,3,4,5,6,7],
    k_per_store: int = 4,
    use_priority_filter: bool = True,
) -> Dict[int, List[Document]]:
    """
    반환: {priority: [docs...]}  (law/rule/case 통합)
    """
    results: Dict[int, List[Document]] = {p: [] for p in priorities}

    # Pinecone 메타 필터 
    # priority 필드는 int
    for p in priorities:
        for store_name, store in vectorstores.items():
            # 필터 구성
            base_filter = {"category": {"$in": categories}}
            if use_priority_filter:
                base_filter["priority"] = {"$eq": p}

            retriever = store.as_retriever(search_kwargs={"k": k_per_store, "filter": base_filter})
            docs = retriever.invoke(query)

            # 출처/인덱스 정보 태깅(나중에 출력/디버깅에 유용)
            for d in docs:
                d.metadata["index"] = store_name
                # priority가 문서 메타에 없다면 p로 보정
                d.metadata.setdefault("priority", p)

            results[p].extend(docs)

        results[p] = dedupe_docs(results[p])

    return results


# ===================================================
# 메인: ask_with_reference (멀티 인덱스 + priority 우선)
# ===================================================

def ask_with_reference(query: str, k: int = 4):
    """
    query, 표준화된 query, 답변, (priority별) 참조사항 출력
    - 답변 컨텍스트: priority=1 문서만 우선 사용
    - 추가참고: priority=2~7 문서가 있으면 함께 출력
    """

    # 1) llm, embedding
    # llm = ChatOpenAI(model=OPENAI_LLM_MODEL)
    # llm = ChatOllama(model="exaone3.5:2.4b", temperature=0.1)
    # llm = ChatOpenAI(model='gpt-4.1-mini')
    # llm = ChatOllama(model="bsahane/Qwen2.5-VL-7B-Instruct:Q4_K_M_benxh", temperature=0.1)
    llm = ChatOllama(model="mervinpraison/llama3.1-instruct:8b", temperature=0.1)
    
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

    # 2) vector DB 연결 (3개 다)
    law_database = PineconeVectorStore(embedding=up_embedding, index_name=PINECONE_INDEX_NAME_LAW)
    rule_database = PineconeVectorStore(embedding=up_embedding, index_name=PINECONE_INDEX_NAME_RULE)
    case_database = PineconeVectorStore(embedding=up_embedding, index_name=PINECONE_INDEX_NAME_CASE)

    vectorstores = {
        "law": law_database,
        "rule": rule_database,
        "case": case_database,
    }

    # 3) keyword_chain 
    keyword_dict = [
        # 사람
        "세입자 -> 임차인",
        "월세입자 -> 임차인",
        "세들어사는사람 -> 임차인",
        "임차자 -> 임차인",
        "입주자 -> 임차인",          

        "집주인 -> 임대인",          
        "임대인 -> 임대인",
        "주인 -> 임대인",
        "건물주 -> 임대인", 
        "임대업자 -> 임대인",
        
        "부동산 -> 공인중개사", 
        "중개인 -> 공인중개사",

        # 금액/지급
        "전세금 -> 임차보증금",  
        "보증금 -> 임차보증금",  
        "월세 -> 차임",    
        "월임대료 -> 차임",
        "임대료 -> 차임",
        "렌트비 -> 차임",
        "방세 -> 차임",
        "관리비 -> 관리비용",

        "보증금돌려받기 -> 보증금 반환",
        "보증금반환 -> 보증금 반환",
        "보증금못받음 -> 보증금 반환",

        # 권리/요건
        "전입신고 -> 주민등록",     
        "전입 -> 주민등록",
        "확정일자 -> 확정일자",    
        "대항요건 -> 대항요건",
        "대항력 -> 대항력",         
        "우선변제 -> 우선변제",
        "우선변제권 -> 우선변제권",

        # 절차/사건
        "조정위 -> 주택임대차분쟁조정위원회",  
        "분쟁조정 -> 주택임대차분쟁조정위원회",
        
        # [행위 - 돈]
        "올려 -> 증액", "인상 -> 증액", "더 달라고 -> 증액", "더 내라고 -> 증액",
        "깎아 -> 감액", "내려 -> 감액", "할인 -> 감액",
        "돌려주지 -> 반환", "안 줘 -> 반환거부", "떼먹 -> 미반환",
        
        # [행위 - 거주/계약]
        "계약서 -> 임대차계약증서", "특약 -> 특약사항",
        "이사 -> 주택의인도", "전입 -> 주민등록/대항력",
        "살고 -> 점유", "거주 -> 점유",
        "나가라 -> 계약갱신거절/명도", "나가라고 -> 계약갱신거절/명도", "비워 -> 명도", "쫓겨 -> 명도/대항력",
        "방 빼 -> 계약해지", "나갈게 -> 계약해지", "해지 -> 계약해지",
        "연장 -> 계약갱신", "더 살 -> 계약갱신", "재계약 -> 계약갱신",
        "수리 -> 수선의무/필요비", "고쳐 -> 수선의무", "물 새 -> 누수", "곰팡이 -> 하자",
        "청소비 -> 원상회복비용", "원상복구 -> 원상회복"
    ]

    prompt = ChatPromptTemplate.from_template(f"""
아래 질문에서 단어만 사전을 기준으로 치환하세요.
문장 구조, 시제, 의문형 여부는 변경하지 마세요.
사전 : {keyword_dict}
질문 : {{question}}
출력은 질문 문장만 하세요.
""")

    keyword_chain = prompt | llm | StrOutputParser()

    # 4) 질문 표준화
    normalized_query = keyword_chain.invoke({"question": query})

    # 5) 카테고리 분류 (원본/표준화 중 택1)
    categories = categorize_content(query)

    # 6) priority 우선 검색: 모든 인덱스(law/rule/case)를 priority=1부터 검색
    #    - k_per_store를 키우면 priority=1 근거가 더 잘 잡힘
    grouped_docs = retrieve_by_priority(
        query=normalized_query,
        vectorstores=vectorstores,
        categories=categories,
        priorities=[1,2,3,4,5,6,7],
        k_per_store=max(2, k // 2) if k else 4,  # 대충 균형
        use_priority_filter=True,
    )

    # 7) 답변 컨텍스트는 priority=1만 우선 사용 (없으면 2,3...로 폴백)
    context_docs = grouped_docs.get(1, [])
    used_priority = 1
    if len(context_docs) == 0:
        for p in [2,3,4,5,6,7]:
            if grouped_docs.get(p):
                context_docs = grouped_docs[p]
                used_priority = p
                break

    # (선택) 답변 컨텍스트 길이 제한: 너무 많으면 상위 k개만
    context_docs = context_docs[:k]

    # 8) 참조조항 추출 (priority별로도 뽑고 싶으면 반복하면 됨)
    referenced_articles_main = extract_articles_from_docs(context_docs)

    # 9) RAG 프롬프트
    template = f"""
당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
사용자 질문에는 법률적 맥락을 돕기 위해 '(법률용어)'가 병기되어 있을 수 있습니다. 이를 참고하여 정확하게 분석하세요.

[지시사항]
1. **근거 기반**: 반드시 [참고 법령]에 기반하여 답변하세요.
2. **입력 분석**: 질문에 포함된 괄호 안의 법률 용어(예: 명도, 대항력 등)를 핵심 키워드로 활용하세요.
3. **답변 구조**:
   - 결론부터 명확히(가능/불가능) 제시
   - 📖 법적 근거(제N조) 상세 설명
   - 사용자 상황에 맞는 조언
4. **🚨 위험 경고**: 불리한 특약이나 법 위반 사항(강행규정 위반)은 강력하게 경고하세요.
5. **❗ 면책**: 법적 효력이 없음을 밝히세요.

[참고 법령] (priority={used_priority} 우선 근거)
{{context}}

질문 : {{query}}
답변 :
"""
    rag_prompt = ChatPromptTemplate.from_template(template)
    prompt_chain = rag_prompt | llm | StrOutputParser()

    result = prompt_chain.invoke({
        "context": format_documents(context_docs),
        "query": query
    })

    # 10) 출력: priority=1 답변 + 추가근거(priority 2~7)
    print("원본 질문 :", query)
    print("표준화된 질문 :", normalized_query)
    print("=" * 90)
    print("\n✅", result)

#     # 메인 근거
#     print("\n📌 메인 근거(priority=%d) :" % used_priority, referenced_articles_main)

#     # 추가 근거(부가적으로 같이 출력)
#     extra_refs = {}
#     for p in [1,2,3,4,5,6,7]:
#         if p == used_priority:
#             continue
#         if grouped_docs.get(p):
#             extra_refs[p] = extract_articles_from_docs(grouped_docs[p])

#     if extra_refs:
#         print("\n🧩 추가 참고(부가 근거) :")
#         for p in sorted(extra_refs.keys()):
#             print(f"  - priority={p} :", extra_refs[p])

    # print("\n❗ 위 답변은 AI에 의해 생성된 답변이므로 약간의 오차가 날 수 있습니다 ❗")


### 모델 비교

In [16]:
%%time
# 사용예시 : exaone3.5:2.4b

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 질문 : 벽지 손상 시 임차인이 전체 수리 비용을 부담하고 계약을 해지한다고 명시되어 있나요?

✅ 답변 : ### 답변 요약
**결론**: 불가능 (법률 위반 가능성)

**법적 근거**:
- **주택임대차보호법 제6조 (임대차의 해지 및 해지의 효력)** 및 **제10조 (임대차계약의 해지 사유)**
  - **제6조**: 임대차계약의 해지 사유 중 하나로 '임대인의 중대한 과실'이 명시되어 있습니다. 벽지 손상과 같은 임차인의 책임 범위를 넘어서는 임대인의 과실이 인정될 경우, 임차인은 계약 해지를 주장할 수 있습니다.
  - **제10조**: 계약 해지 시 임대인은 임차인에게 적절한 보상을 제공해야 합니다. 단순히 임차인이 모든 수리 비용을 부담하고 계약을 종료하는 것은 임대인의 의무 이행 범위를 벗어난 경우가 많습니다.

**상세 설명**:
- **임대인의 책임**: 벽지 손상과 같은 문제는 일반적으로 임대인의 관리 부실이나 부주의로 인한 경우가 많습니다. 만약 임대인이 이러한 유지보수 책임을 충분히 이행하지 않았다면, 임차인은 계약 해지를 주장할 수 있습니다.
- **임차인의 권리**: 임차인은 계약 조건에 명시된 책임 범위 내에서만 수리 비용을 부담해야 합니다. 만약 계약서에 명시된 내용이 법률상 허용되는 범위를 벗어난다면, 이는 법적으로 문제가 될 수 있습니다.

**사용자 상황에 맞는 조언**:
- **법률 검토**: 계약서 내용을 법률 전문가와 상의하여 현재 조항의 적법성을 확인하세요. 특히 임대인의 책임 범위와 임차인의 의무를 명확히 구분하는 것이 중요합니다.
- **합의 시도**: 임대인과의 직접적인 협의를 통해 합리적인 해결책을 모색하세요. 예를 들어, 일부 비용 분담이나 수리 범위 조정 등이 고려될 수 있습니다.

**위험 경고**:
- **강행규정 위반**: 계약서에 명시된 내용이 법률상 강행규정 위반으로 간주될 경우, 임차인

In [18]:
%%time
# 사용예시 : gpt-4o-mini

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 계약서에 벽지 등이 손상될 경우 임차인이 전부 수리하고 나간다라고 되어있어요

✅ 답변 : 결론: 불가능

📖 법적 근거:
주택임대차보호법 제8조에 따르면, 임차인은 계약 기간 중 정상적인 사용으로 인한 물건의 자연적 마모나 손상에 대해 책임을 지지 않습니다. 또한, 민법 제618조에서는 임대인의 사용과 수익에 대한 권리와 의무를 규정하고 있으며, 임대인은 일반적으로 임차인이 사용 중 생긴 정상적인 손상에 대해 책임지지 않도록 돕는 의무가 있습니다.

사용자 상황에 맞는 조언:
계약서에 “세입자가 전부 수리하고 나간다”는 조항이 포함되어 있는 경우, 이는 법에 저촉될 수 있습니다. 특히, 벽지와 같은 일반적인 마모에 대한 수리에 대해 세입자가 책임을 지는 것은 불리한 특약으로 해석될 수 있으며, 이는 법적으로 인정되지 않을 가능성이 높습니다. 계약서의 해당 조항이 강행규정에 위반된다면, 법원에서 무효로 판단될 수도 있습니다.

🚨 위험 경고: 계약서에 부적절한 특약이 포함된 경우, 세입자에게 불리한 결과를 초래할 수 있으며, 이는 법적으로 불리한 상황을 만들 수 있습니다. 꼭 법률 전문가와 상담하여 계약서의 내용을 조정하거나 검토받는 것이 좋습니다.

❗ 면책: 본 답변은 법률적 조언이 아니며, 법적 효력이 없습니다. 정확한 법률 상담을 위해 변호사와 상담하시기 바랍니다.

📌 메인 근거(priority=1) : 주택임대차보호법
CPU times: total: 875 ms
Wall time: 26.1 s


In [20]:
%%time
# 사용예시 : gpt-4.1-mini

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 임대차계약증서에 벽지 등이 손상될 경우 임차인이 전부 수선의무하고 주택의인도한다고 되어있어요

✅ 답변 : 결론:  
계약서에 ‘벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다’는 특약 조항은 일부 인정되나, 세입자에게 지나치게 과도한 수리 의무를 부과하는 경우(통상적인 사용으로 인한 손상 포함)에는 법적으로 무효 또는 제한될 수 있습니다. 따라서 전부 수리 의무가 무조건적으로 적용된다고 보기 어렵습니다.

📖 법적 근거:  
- **민법 제623조(임대인의 수리의무 등)**: 임대차 목적물의 관리·수리는 원칙적으로 임대인이 부담합니다. 다만 과실이나 고의로 인한 손상에 한해 임차인이 배상할 책임이 있습니다.  
- **주택임대차보호법 제3조(대항력)**: 임차인의 주거 안정 보호 차원에서 통상적인 사용에 따른 손상이나 마모는 임차인 책임으로 보기 어렵습니다.  
- 대법원 판례에 따르면, 통상적인 사용으로 인한 벽지 변색, 노후된 벽지 손상에 대해 세입자에게 전부 수리를 요구하는 것은 부당할 수 있습니다.

사용자 상황에 맞는 조언:  
- 계약서의 ‘전부 수리’ 조항이 통상적인 사용으로 인한 손상까지 포함하여 과도하게 해석될 경우, 임차인에게 불리하게 작용할 수 있으므로 주의해야 합니다.  
- 입주 시 벽지 상태를 상세히 사진 등으로 기록하여 ‘기존 손상’과 ‘임차인 책임 손상’을 구분하세요.  
- 과도한 수리 부담이 예상될 경우 임대인과 협의하거나 법률 상담을 받는 것이 권장됩니다.

🚨 위험 경고:  
- 계약서에 통상적인 마모나 노후에 대한 수리 책임까지 임차인에게 전가하는 것은 **민법과 주택임대차보호법상 강행규정 위반이 될 수 있으므로 매우 위험**합니다.  
- 불리한 특약은 분쟁 시 무효 또는 제한될 가능성이 높으니 계약 전 면밀히 검토하시기 바랍니다.

❗ 면책:  
본 답변은 참고용이며, 구체 상황에 따라 법원의 판단이 달라

In [24]:
%%time
# 사용예시 : bsahane/Qwen2.5-VL-7B-Instruct:Q4_K_M_benxh

ask_with_reference('계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요')

원본 질문 : 계약서에 벽지 등이 손상될 경우 세입자가 전부 수리하고 나간다라고 되어있어요
표준화된 질문 : 계약서에 벽지 등이 손상될 경우 임차인이 전부 수리하고 나간다고 되어있어요.

✅ 답변 : **결론**: 불가능합니다.

**법적 근거**: **주택임대차보호법 제31조**

**해석 및 조언**: 주택임대차보호법 제31조에 따르면, 임대인이 임대차 계약서에서 세입자가 전부 수리하고 나간다는 내용을 명시하는 것은 불법입니다. 이는 강제규정 위반이며, 법적으로 무효로 간주됩니다.

**주의사항**: 이 조항은 임대인과 세입자 모두에게 적용되므로, 해당 내용이 포함된 계약서는 무효로 처리되어야 합니다. 이를 수정하거나 재협의하여 합법적인 내용으로 변경해야 합니다. 만약 이에 대한 불만이 있다면, 전문 변호사와 상담을 통해 법률적 조언을 받으시기 바랍니다.

**주의사항**: 위 내용은 참고 법령에 기반한 분석이며, 실제 법률 적용 시에는 다양한 요소가 고려될 수 있으므로, 상황에 따라 변동이 있을 수 있습니다.
CPU times: total: 641 ms
Wall time: 2min 27s
